In [3]:
import pandas as pd

In [4]:
df_train = pd.read_csv("train_prep.csv")

In [5]:
drop_columns = ['Unnamed: 0.1', 'Unnamed: 0']
df_train.drop(drop_columns,axis=1,inplace=True)

In [6]:
df_train.columns

Index(['Month', 'Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment',
       'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix',
       'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [7]:
#Split Input & Output Data
X = df_train.drop('Credit_Score',axis=1)
y = df_train['Credit_Score']
print(X.shape)
print(y.shape)

(100000, 23)
(100000,)


In [8]:
from sklearn.model_selection import train_test_split,cross_val_score

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17,stratify=y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(80000, 23)
(20000, 23)
(80000,)
(20000,)


In [10]:
from sklearn.metrics import make_scorer, precision_score, recall_score

# Custom scorer for precision of the minority class
precision_minority_scorer = make_scorer(precision_score, pos_label=1)

# Custom scorer for precision of the majority class
precision_majority_scorer = make_scorer(precision_score, pos_label=0)

# Custom scorer for recall of the minority class
recall_minority_scorer = make_scorer(recall_score, pos_label=1)


In [11]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

# Initialize the model
classifier = RandomForestClassifier()

# Define the cross-validation configuration
cv = StratifiedKFold(5)

# Initialize tracking for the best scores and best features for each metric
best_scores = {
    'precision_minority': 0,
    'precision_majority': 0,
    'recall_minority': 0
}
best_features = {
    'precision_minority': None,
    'precision_majority': None,
    'recall_minority': None
}

# Start with all features
features = list(X_train.columns)
while features:
    # Subset the training data to the current set of features
    X_train_subset = X_train[features]

    # Perform cross-validation
    scores = cross_val_score(classifier, X_train_subset, y_train, cv=cv, scoring=precision_minority_scorer)
    precision_minority = np.mean(scores)
    scores = cross_val_score(classifier, X_train_subset, y_train, cv=cv, scoring=precision_majority_scorer)
    precision_majority = np.mean(scores)
    scores = cross_val_score(classifier, X_train_subset, y_train, cv=cv, scoring=recall_minority_scorer)
    recall_minority = np.mean(scores)

    # Check if current feature set is the best for any metric
    if precision_minority > best_scores['precision_minority']:
        best_scores['precision_minority'] = precision_minority
        best_features['precision_minority'] = features.copy()
    if precision_majority > best_scores['precision_majority']:
        best_scores['precision_majority'] = precision_majority
        best_features['precision_majority'] = features.copy()
    if recall_minority > best_scores['recall_minority']:
        best_scores['recall_minority'] = recall_minority
        best_features['recall_minority'] = features.copy()

    # Remove the least important feature (simple method: last in list for illustration)
    if features:
        features.pop()

# Output the best scores and corresponding features
print("Best Scores:", best_scores)
print("Best Features for Precision (Minority Class):", best_features['precision_minority'])
print("Best Features for Precision (Majority Class):", best_features['precision_majority'])
print("Best Features for Recall (Minority Class):", best_features['recall_minority'])


c:\Users\aditya.singh1\Desktop\Credit-scoring-model\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aditya.singh1\Desktop\Credit-scoring-model\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aditya.singh1\Desktop\Credit-scoring-model\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

Best Scores: {'precision_minority': np.float64(0.8181799846945262), 'precision_majority': np.float64(0.9283276437535107), 'recall_minority': np.float64(0.8260621343779502)}
Best Features for Precision (Minority Class): ['Month', 'Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan']
Best Features for Precision (Majority Class): ['Month', 'Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age']
Best Features for Recall (Minority Class): ['Month', 'Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_

: 